In [2]:
import request_category as rc
import database as db
from download import download

import pickle
from pathlib import Path
import re

import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline

import search


from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelBinarizer, LabelEncoder

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, StratifiedShuffleSplit
from sklearn.linear_model import LogisticRegression
import sklearn.externals.joblib

## Test out multinomial for all subcategories

In [3]:
pages_df = search.get_data( 'machine learning', 'business software')

pages_df.shape

(2756, 4)

In [4]:
pages_df.head()

,article,category,subcategory,title
pageid,,,,
49076372,concerto project management web based project ...,business software,Project management software,concerto project management
44738576,machine learning hyper basis function network ...,machine learning,Classification algorithms,hyper basis function network
41200806,proximal gradient forward backward splitting m...,machine learning,machine learning,proximal gradient methods for learning
1863114,sap netweaver primary technology computing pla...,business software,Service-oriented architecture-related products,sap netweaver
41667831,financial software financial system software s...,business software,Financial software,financial software


In [5]:
X = pages_df['article'].copy()

encoder = LabelEncoder()
y = encoder.fit_transform( pages_df['subcategory'] )

X_train, X_test, y_train, y_test = train_test_split( X, y, random_state = 42)

In [ ]:
lr_pipe = Pipeline([
    ('encoder', TfidfVectorizer(ngram_range = (1,2),
                             min_df = 3, max_df = .9, 
                             stop_words = 'english')),
    ('truncator',TruncatedSVD(n_components=700) ),
    ('model', LogisticRegression())
])

lr_params = {
    'model__C':np.logspace(-3,3,7)
}

gs_lr_pipe = GridSearchCV(lr_pipe, param_grid=lr_params, cv=5) # StratifiedShuffleSplit(n_splits=5)

gs_lr_pipe.fit(X_train,y_train)

gs_lr_pipe.best_score_

/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:581: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of groups for any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


In [8]:
sklearn.externals.joblib.dump(gs_lr_pipe, './LogitModel1.p')

['./LogitModel1.p']

In [114]:
search_doc, pageid = rc.get_article('Saffron Technology')  # Slack (software) Saffron Technology  Brain Statistics  TensorFlow Tableau Software
search_doc = rc.cleaner( search_doc)  # Tableau Software
##print( pageid )
search_doc[0:200]

'saffron technology technology company headquartered cary north carolina develops cognitive computing systems use incremental learning understand unify entity person place thing connections entity thin'

In [115]:
predicted_probs = gs_lr_pipe.predict_proba([search_doc])  ## 'Brain'
#predicted_probs.shape
test = predicted_probs.reshape(-1,1)#.shape#.shape  test = 

In [116]:
cats = encoder.inverse_transform(  range(0,63))

probs_df = pd.DataFrame( test , columns= ['P'], index = cats)

probs_df.sort_values(by = ['P'],ascending=False)

,P
business software,0.679407
Data mining and machine learning software,0.080706
Artificial neural networks,0.029073
Business software companies,0.025276
machine learning,0.017790
Collaborative software,0.013535
Machine learning researchers,0.011528
Human resource management software,0.010159
Applied machine learning,0.008394
Portal software,0.007276


In [108]:
max( y)

62

In [60]:
test.shape

(1, 63)

In [33]:
encoder.inverse_transform( gs_lr_pipe.predict([search_doc]))

array(['machine learning'], dtype=object)

In [32]:
gs_lr_pipe.predict([search_doc])

array([62])

array(['Administrative software', 'Applied machine learning',
       'Artificial intelligence conferences', 'Artificial neural networks',
       'Bayesian networks', 'Business simulation games',
       'Business software companies', 'Business software for Linux',
       'Business software for MacOS', 'Business software for Windows',
       'Business software stubs', 'Classification algorithms',
       'Cluster analysis', 'Collaborative software',
       'Computational learning theory',
       'Data mining and machine learning software',
       'Datasets in machine learning',
       'Dental practice management software', 'Dimension reduction',
       'ERP software', 'Ensemble learning', 'Enterprise software',
       'Evolutionary algorithms', 'Financial software',
       'Free business software', 'Genetic programming', 'Health software',
       'Healthcare software', 'Human resource management software',
       'Inductive logic programming',
       'Industry-specific XML-based standards